In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
!pip install ipywidgets
!pip install opendatasets ipyopenlayers pandas

from ipyopenlayers import RasterTileLayer, ZoomSlider, Map
from ipyleaflet import Map, Marker, MarkerCluster,Popup
from ipywidgets import HTML
import opendatasets as od
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import time
import os
import kagglehub
from tqdm import tqdm
import pickle

!pip install pyspark opendatasets kagglehub

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, avg, when, lit, min as spark_min, max as spark_max

spark = SparkSession.builder \
    .appName("PollutionDataProcessing") \
    .getOrCreate()


import kagglehub
import zipfile

# # !kaggle datasets download -d guslovesmath/us-pollution-data-200-to-2022 -p ./ --unzip
# path = kagglehub.dataset_download("guslovesmath/us-pollution-data-200-to-2022")
# with zipfile.ZipFile("us-pollution-data-200-to-2022.zip", 'r') as zip_ref:
#     zip_ref.extractall("data")
file_path = "pollution_2000_2023.csv"




In [ ]:
file_path = "pollution_2000_2023.csv"



df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)
df.printSchema()
df.select("Date", "State", "City", "NO2 1st Max Value").show(5)

def calculate_no2_aqi_spark():
    return when(col("NO2 1st Max Value") <= 53,  ((50 - 0) / (53 - 0)) * (col("NO2 1st Max Value") - 0) + 0)\
        .when((col("NO2 1st Max Value") <= 100), ((100 - 51) / (100 - 54)) * (col("NO2 1st Max Value") - 54) + 51)\
        .when((col("NO2 1st Max Value") <= 360), ((150 - 101) / (360 - 101)) * (col("NO2 1st Max Value") - 101) + 101)\
        .when((col("NO2 1st Max Value") <= 649), ((200 - 151) / (649 - 361)) * (col("NO2 1st Max Value") - 361) + 151)\
        .when((col("NO2 1st Max Value") <= 1249), ((300 - 201) / (1249 - 650)) * (col("NO2 1st Max Value") - 650) + 201)\
        .otherwise(((500 - 301) / (2049 - 1250)) * (col("NO2 1st Max Value") - 1250) + 301)

df = df.withColumn("NO2 AQI", calculate_no2_aqi_spark())




root
 |-- _c0: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Address: string (nullable = true)
 |-- State: string (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- O3 Mean: double (nullable = true)
 |-- O3 1st Max Value: double (nullable = true)
 |-- O3 1st Max Hour: integer (nullable = true)
 |-- O3 AQI: integer (nullable = true)
 |-- CO Mean: double (nullable = true)
 |-- CO 1st Max Value: double (nullable = true)
 |-- CO 1st Max Hour: integer (nullable = true)
 |-- CO AQI: double (nullable = true)
 |-- SO2 Mean: double (nullable = true)
 |-- SO2 1st Max Value: double (nullable = true)
 |-- SO2 1st Max Hour: integer (nullable = true)
 |-- SO2 AQI: double (nullable = true)
 |-- NO2 Mean: double (nullable = true)
 |-- NO2 1st Max Value: double (nullable = true)
 |-- NO2 1st Max Hour: integer (nullable = true)
 |-- NO2 AQI: integer (nullable = true)

+----------+-------+-------+-----------------+
|      Date|  State|   Ci

In [ ]:
from pyspark.sql.functions import avg, substring, lit

# Extragem anul și luna
df = df.withColumn("Year", substring("Date", 1, 4).cast("int"))
df = df.withColumn("Month", substring("Date", 6, 2).cast("int"))

# Alegem coloanele relevante
selected_columns = [
    "Date", "State", "County", "City", "O3 Mean", "O3 1st Max Value", "O3 AQI",
    "CO Mean", "CO 1st Max Value", "CO AQI",
    "SO2 Mean", "SO2 1st Max Value", "SO2 AQI",
    "NO2 Mean", "NO2 1st Max Value", "NO2 AQI", "Year", "Month"
]
filtered_df = df.select(*selected_columns)



In [ ]:
print(filtered_df.head())


Row(Date=datetime.date(2000, 1, 1), State='Arizona', County='Maricopa', City='Phoenix', O3 Mean=0.019765, O3 1st Max Value=0.04, O3 AQI=37, CO Mean=0.8789469999999999, CO 1st Max Value=2.2, CO AQI=25.0, SO2 Mean=3.0, SO2 1st Max Value=9.0, SO2 AQI=13.0, NO2 Mean=19.041667, NO2 1st Max Value=49.0, NO2 AQI=46.22641509433962, Year=2000, Month=1)


In [ ]:
# Coloane de grupare
group_cols_month = ["Year", "Month", "State", "County", "City"]
group_cols_year = ["Year", "State", "County", "City"]

# Coloane care vor fi agregate (exclude toate coloanele de grupare)
agg_cols = [col for col in filtered_df.columns if col not in group_cols_month + ["Date"]]

# Agregare pe lună
agg_month = filtered_df.groupBy(*group_cols_month) \
    .agg(*[avg(col_name).alias(col_name) for col_name in agg_cols]) \
    .withColumn("Granularity", lit("monthly"))

# Agregare pe an (Month = All)
agg_year = filtered_df.groupBy(*group_cols_year) \
    .agg(*[avg(col_name).alias(col_name) for col_name in agg_cols]) \
    .withColumn("Month", lit(None)) \
    .withColumn("Granularity", lit("yearly"))

# Union corect
aggregated_df = agg_month.unionByName(agg_year)


In [ ]:
print(agg_month.head())


Row(Year=2000, Month=7, State='California', County='Sacramento', City='Arden-Arcade', O3 Mean=0.03446293548387097, O3 1st Max Value=0.05558064516129033, O3 AQI=69.48387096774194, CO Mean=0.17597754838709673, CO 1st Max Value=0.2806451612903226, CO AQI=3.0, SO2 Mean=1.93494964516129, SO2 1st Max Value=5.258064516129032, SO2 AQI=7.483870967741935, NO2 Mean=10.871969548387096, NO2 1st Max Value=17.967741935483872, NO2 AQI=16.950699939135724, Granularity='monthly')


In [ ]:

# Combinare
aggregated_df = agg_month.unionByName(agg_year)

# Ranking ca sumă AQI
aggregated_df = aggregated_df.withColumn("Ranking",
    col("O3 AQI") + col("CO AQI") + col("SO2 AQI") + col("NO2 AQI")
)

from pyspark.sql.window import Window
from pyspark.sql.functions import min, max

w = Window.partitionBy("Year", "Month")

aggregated_df = aggregated_df.withColumn("min_rank", spark_min("Ranking").over(w)) \
    .withColumn("max_rank", spark_max("Ranking").over(w)) \
    .withColumn("ranking_final_value", 1 - ((col("Ranking") - col("min_rank")) / (col("max_rank") - col("min_rank")))) \
    .drop("min_rank", "max_rank")

aggregated_df.write.mode("overwrite").option("header", True).csv("aggregated_pollution_data_spark.csv")


In [ ]:
aggregated_df_pd = aggregated_df.toPandas()
aggregated_df_pd.to_csv('aggregated_pollution_data_spark_final_all.csv', index=False)

In [ ]:
# Set up geolocator with increased timeout
geolocator = Nominatim(user_agent="pollution_map_app", timeout=10)

# Load cache if it exists
cache_file = "geocode_cache.pkl"
if os.path.exists(cache_file):
    with open(cache_file, "rb") as f:
        geocode_cache = pickle.load(f)
else:
    geocode_cache = {}

# Create a dictionary to store locations not found
not_found_cache = {}

# Define geocoding function with caching
def geocode_location(row):
    city_state = f"{row['City']}, {row['State']}"
    if city_state in geocode_cache:
        return geocode_cache[city_state]
    if city_state in not_found_cache:
        return None

    try:
        if row['City'] == 'Not in a city':
            # Geocode using only state
            location = geolocator.geocode({"state": row['State'], "country": "USA"})
            if location:
                point = Point(location.longitude, location.latitude)
                geocode_cache[city_state] = point
                return point
            else:
                print(f"Location not found for state: {row['State']}")
                not_found_cache[city_state] = None
                return None
        else:
            # Geocode using city and state
            location = geolocator.geocode({"city": row['City'], "state": row['State'], "country": "USA"})
            if location:
                point = Point(location.longitude, location.latitude)
                geocode_cache[city_state] = point
                return point
            else:
                print(f"Location not found: {city_state}")
                # Attempt to geocode using only state
                location2 = geolocator.geocode({"state": row['State'], "country": "USA"})
                if location2:
                    point = Point(location2.longitude, location2.latitude)
                    geocode_cache[city_state] = point
                else:
                    not_found_cache[city_state] = None
                return None
    except Exception as e:
        print(f"Geocoding error for {city_state}: {e}")
        time.sleep(1)  # Delay to handle rate limits
        return None

# Apply geocoding with a progress bar
aggregated_df_pd = aggregated_df.toPandas()
aggregated_df_pd['geometry'] = [geocode_location(row) for _, row in tqdm(aggregated_df_pd.iterrows(), total=len(aggregated_df_pd))]

# Save cache for future use
with open(cache_file, "wb") as f:
    pickle.dump(geocode_cache, f)

# Save not found cache for future reference
not_found_cache_file = "not_found_cache.pkl"
with open(not_found_cache_file, "wb") as f:
    pickle.dump(not_found_cache, f)

# Drop rows with missing geometry
df = aggregated_df_pd.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(aggregated_df_pd, geometry='geometry', crs="EPSG:4326")

  0%|          | 17/22798 [00:16<6:17:05,  1.01it/s]

Location not found: Lexington-Fayette (corporate name for Lexington), Kentucky


  0%|          | 52/22798 [00:52<6:15:38,  1.01it/s]

Location not found: Vandenberg Air Force Base, California


  0%|          | 67/22798 [01:08<6:20:50,  1.01s/it]

...

  0%|          | 71/22798 [01:13<7:01:25,  1.11s/it]

Location not found: Calumet City (PU RR name Calumet Park (sta.)), Illinois


 34%|███▍      | 7793/22798 [02:46<00:44, 335.69it/s]

Location not found: Dentsville (Dents), South Carolina


100%|██████████| 22798/22798 [02:55<00:00, 130.11it/s] 


In [ ]:
gdf.to_csv("pollution_data_spark_geocode.csv", index=False)
